# Row Filtering

## Learning Objectives

In this tutorial, we will demonstrate how to:

- Set up a Dask client and open an object catalog
- Filter rows of data by expressions involving column values
- Do quick previews of catalog data and query results

## Introduction

When a catalog is opened, it is available for operations.  However, its data is is lazily loaded, and operations on it are unrealized, until computation is called for explicitly (using the `.compute()` method) or implicitly, with data preview functions.

In [ ]:
import lsdb
from dask.distributed import Client

## 1. Open a catalog

We create a basic dask client, and open an existing HATS catalog—the ZTF DR14 catalog.

Create a basic Dask client, limiting the number of workers. This keeps subsequent operations from using more of our compute resources than we might intend, which is helpful in any case but especially when working on a shared resource.

In [ ]:
client = Client(n_workers=4, memory_limit="auto")
client

In [ ]:
ztf_object_path = "https://data.lsdb.io/hats/ztf_dr14/ztf_object"
ztf_object = lsdb.open_catalog(ztf_object_path)
ztf_object

### 1.1. Previewing part of the data

Computing an entire catalog requires loading all of its resulting data into memory, which is expensive and may lead to out-of-memory issues. 

Often, our goal is to have a peek at a slice of data to make sure the workflow output is reasonable (e.g., to assess if some new created columns are present and their values have been properly processed). `head()` is a Pandas-like method which allows us to preview part of the data for this purpose. It iterates over the existing catalog partitions, in sequence, and finds up to `n` number of rows from the first partition(s) which have are able to supply those rows. Related methods include `.tail()` and `.sample()`.

There is also `.random_sample()`, but that method fetches rows from many partitions (rather than from first qualified), and so it can be much more expensive, even while it may be more representative.

Notice that all these previewing methods implicitly call `compute()`, and will implicitly use the `Client` we created earlier.

In [ ]:
ztf_object.head()

## 2. Selecting data rows by querying column values

We can filter by column values via `query()`.

The expression in the string given to `.query()` follows the same syntax accepted by Pandas' `.query()`, which supports a subset of Python expressions for filtering DataFrames.

The column names that are not valid Python variables names should be wrapped in backticks, and any variable values can be injected using f-strings. The use of '@' to reference variables is not supported.

More information about Pandas query strings is available [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html).

In the following query, we want to find objects in the catalog whose magnitude is brighter than 16:

In [ ]:
bright = ztf_object.query("mean_mag_i < 16")
bright

We'll use `.head()` for a quick sanity check to be sure that no `mean_mag_i` is dimmer than 16. Since it's only a few rows, it's not a guarantee, but it does help us to be sure that we didn't make any obvious mistake with our query expression.

In [ ]:
%%time
bright.head(10)

You can use parentheses, logical operators, and more than one column name in your expressions. Here, we alter the query to include not only those objects with a `mean_mag_i` that is brighter than 16, but which have at least 50 observations in that band. Note that this query takes longer than the original, mostly because it takes longer to find rows that satisfy this stricter query.

In [ ]:
%%time
bright_hi_obs = ztf_object.query("mean_mag_i < 16 and nobs_i > 50")
bright_hi_obs.head(10)

## 4. Filtering using Python expressions

In some cases it may be more readable to query using Python expressions, Pandas-style. In this form, the catalog is indexed using an expression, selecting the rows for which the expression is true. The form of this query is `filtered = collection[expr_with_collection]`, where `expr_with_collection` needs to evaluate to something which is:

  * of the same size as `collection`; and
  * convertible to boolean

The below expression produces the same result as the earlier `.query()` example, and whether it is more tractable than `.query()` depends on your expression and what it includes. But there are a couple of fixes we need to make, things which `.query()` does for you.

  * The use of `&` instead of `and` (also, `|` vs. `or`). The Python logicals don't work here.
  * Having to use `(` and `)` to ensure the intended precedence of the operators. (`&` and `|` are bitwise operators and, without parentheses, bind higher than the logical operators `and` and `or`.)

Note that the time taken is basically identical to that of the `.query` method. There is no particular performance advantage to either approach, as the underlying computations are vectorized the same way.

In [ ]:
%%time
bright_ex = ztf_object[(ztf_object["mean_mag_i"] < 16) & (ztf_object["nobs_i"] > 50)]
bright_ex.head()

## Closing the Dask client

In [ ]:
client.close()

## About

**Authors**: Sandro Campos, Melissa DeLucchi, Olivia Lynn, and Derek Jones

**Last updated on**: April 14, 2025

If you use `lsdb` for published research, please cite following [instructions](https://docs.lsdb.io/en/stable/citation.html).